In [ ]:
#ФИО:Михайлова Ксения   Домашнее задание (Ансамблирование)

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import auc, roc_curve, roc_auc_score
%matplotlib inline

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [5]:
#dell min row

In [5]:
target = 'SalePrice'

In [6]:
y = data[target]

In [7]:
X = data.drop([target], axis=1)

In [8]:
X = X.drop(['Id','Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)

In [9]:
cat_feat = list(X.dtypes[X.dtypes == object].index)

In [10]:
X[cat_feat] = X[cat_feat].fillna('nan')

In [11]:
#отфильтруем непрерывные признаки
num_feat = [f for f in X if f not in (cat_feat + ['ID'])]

In [12]:
cat_nunique = X[cat_feat].nunique()
print(cat_nunique)

MSZoning          5
Street            2
LotShape          4
LandContour       4
Utilities         2
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        8
BldgType          5
HouseStyle        8
RoofStyle         6
RoofMatl          8
Exterior1st      15
Exterior2nd      16
MasVnrType        5
ExterQual         4
ExterCond         5
Foundation        6
BsmtQual          5
BsmtCond          5
BsmtExposure      5
BsmtFinType1      7
BsmtFinType2      7
Heating           6
HeatingQC         5
CentralAir        2
Electrical        6
KitchenQual       4
Functional        7
GarageType        7
GarageFinish      4
GarageQual        6
GarageCond        6
PavedDrive        3
SaleType          9
SaleCondition     6
dtype: int64


In [13]:
cat_feat = list(cat_nunique[cat_nunique < 10].index)

In [14]:
# Создаем дамми-переменные для категорий
dummy_X = pd.get_dummies(X[cat_feat], columns=cat_feat)

In [15]:
dummy_cols = list(set(dummy_X))

In [16]:
dummy_X = dummy_X[dummy_cols]

In [17]:
dummy_X.head()

,SaleType_ConLw,ExterQual_TA,BsmtExposure_nan,Heating_OthW,GarageFinish_Fin,ExterCond_Fa,BsmtQual_Gd,MSZoning_RM,BldgType_Twnhs,Functional_Maj1,...,RoofMatl_Roll,BsmtQual_TA,PavedDrive_Y,SaleType_ConLD,SaleType_WD,Condition2_PosN,BsmtExposure_Gd,HeatingQC_Ex,Functional_Min2,HouseStyle_1.5Unf
0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,1,0,0
1,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,1,1,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,1,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,1,0,0


In [18]:
X_fin = pd.concat([X[num_feat].fillna(-999), dummy_X], axis=1)

In [19]:
X_fin.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,RoofMatl_Roll,BsmtQual_TA,PavedDrive_Y,SaleType_ConLD,SaleType_WD,Condition2_PosN,BsmtExposure_Gd,HeatingQC_Ex,Functional_Min2,HouseStyle_1.5Unf
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,1,0,1,0,0,1,0,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,1,0,1,0,1,1,0,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,1,0,1,0,0,1,0,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,1,1,0,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,1,0,1,0,0,1,0,0


In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split( X_fin, y, test_size=0.3, random_state=42)

In [19]:
#k-fold

In [22]:
from sklearn.model_selection import KFold, cross_val_score

In [23]:
kf = KFold(n_splits=10)

In [21]:
#randomforest

In [24]:
from sklearn.ensemble import RandomForestRegressor  

In [25]:
clf_rfr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)

In [26]:
clf_rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [27]:
y_pred_test_rf2 = clf_rfr.predict(X_test)
y_pred_train_rf2 = clf_rfr.predict(X_train)

In [28]:
imp2 = pd.Series(clf_rfr.feature_importances_)
imp2.sort_values(ascending=False)

3      0.808255
15     0.095201
13     0.038753
125    0.016265
12     0.008565
8      0.008203
25     0.007255
2      0.004233
11     0.004164
206    0.004122
18     0.002584
32     0.001480
22     0.000921
79     0.000000
80     0.000000
77     0.000000
81     0.000000
82     0.000000
76     0.000000
75     0.000000
83     0.000000
78     0.000000
224    0.000000
74     0.000000
73     0.000000
72     0.000000
85     0.000000
71     0.000000
70     0.000000
69     0.000000
         ...   
113    0.000000
138    0.000000
140    0.000000
166    0.000000
154    0.000000
165    0.000000
164    0.000000
163    0.000000
162    0.000000
161    0.000000
160    0.000000
159    0.000000
158    0.000000
157    0.000000
156    0.000000
155    0.000000
153    0.000000
141    0.000000
152    0.000000
151    0.000000
150    0.000000
149    0.000000
148    0.000000
147    0.000000
146    0.000000
145    0.000000
144    0.000000
143    0.000000
142    0.000000
0      0.000000
Length: 225, dtype: floa

In [29]:
X_train.iloc[[1], [0,3,13, 15] ]

,MSSubClass,OverallQual,2ndFlrSF,GrLivArea
1452,180,5,0,1072


In [30]:
X_f = X_fin[['OverallQual','GrLivArea']]

In [31]:
X_train, X_test, y_train, y_test = train_test_split( X_f, y, test_size=0.3)

In [27]:
#Стекинг

In [ ]:
#ver/2

In [71]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, BayesianRidge
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

In [ ]:
#linear

In [74]:
clf_l = Lasso(alpha=0.0005)
clf_r = Ridge(alpha=1.0)

In [51]:
#nonlinear

In [52]:
clf_svr = SVR(kernel='linear', C=10, gamma=0.001)

In [75]:
scores = cross_val_score(clf_l, X_f, y, cv=kf, scoring='r2')
print(scores.mean(), 'Lasso' )

0.7139384796364051 Lasso


In [76]:
scores = cross_val_score(clf_r, X_f, y, cv=kf, scoring='r2')
print(scores.mean(), 'Ridge' )

0.7139440156140769 Ridge


In [64]:
scores = cross_val_score(clf_svr, X_f, y, cv=kf, scoring='r2')
print(scores.mean(), 'SVR' )

0.5767076154776938 SVR


In [37]:
from sklearn.model_selection import StratifiedKFold

In [38]:
folds=StratifiedKFold(n_splits=10,random_state=555)

In [39]:
X_train2, X_test2, y_train2, y_test2 = train_test_split( X_f, y, test_size=0.3)

In [40]:
clf_l2 = Lasso(alpha=0.1)
clf_r2 = Ridge(alpha=1.0)
clf_svr2 = SVR(kernel='linear', C=10, gamma=0.001)

In [41]:
def get_meta_features(clf, X_train, y_train, X_test, stack_cv):
    meta_train = np.zeros_like(y_train, dtype=float)
    meta_test = np.zeros_like(y_test, dtype=float)
    
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        
        clf.fit(X_train.iloc[train_ind], y_train.iloc[train_ind])
        meta_train[test_ind] = clf.predict(X_train.iloc[test_ind])
        meta_test += clf.predict(X_test)
    
    return meta_train, meta_test / stack_cv.n_splits

In [42]:
meta_train = []
meta_test = []
col_names = []

print('Lasso features...')
meta_tr, meta_te = get_meta_features(clf_l2, X_train2, y_train2, X_test2, folds)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('lasso_pred')

print('Ridge features...')
meta_tr, meta_te = get_meta_features(clf_r2, X_train2, y_train2, X_test2, folds)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('ridge_pred')

print('SVR features...')
meta_tr, meta_te = get_meta_features(clf_svr2, X_train2, y_train2, X_test2, folds)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('SVR_pred')

Lasso features...
Ridge features...


/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


SVR features...


/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [43]:
X_meta_train = pd.DataFrame(np.stack(meta_train, axis=1), columns=col_names)
X_meta_test = pd.DataFrame(np.stack(meta_test, axis=1), columns=col_names)

In [77]:
meta_clf_KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [78]:
meta_clf_KRR.fit(X_meta_train, y_train2)

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:167: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


KernelRidge(alpha=0.6, coef0=2.5, degree=2, gamma=None, kernel='polynomial',
      kernel_params=None)

In [79]:
meta_clf_KRR.score(X_meta_test, y_test2)

0.6723226794022097